# 期末專題發想：自動將影片進行分類

之前看過一篇文章，是在說youtube上曾經有一些影片是利用一些小朋友喜歡的角色(例如：蜘蛛人、冰雪奇緣...等)，但是劇情卻安插血腥、暴力、情色的內容，這樣的影片會透過"自動撥放"的功能，讓小朋友看到。所以，我希望能有一個方法能自動將影片分類。


* 資料來源：AudioSet (https://research.google.com/audioset/index.html)
* 網站中，從YouTube中收集2,084,320個有標記類別的10秒聲音片段的影片。包含的類別有：音樂、演講、引擎發動聲、鳥鳴...等，527個類別
* 提供的資料有兩種：
    * csv檔：影片在YouTube上的ID、影片開始時間、影片結束時間、影片類別(可能是多個類別)
    * .tfrecord格式的檔案：將10秒鐘的影片，每秒取出128维的特徵。特徵是使用VGGish模型来提取的。
* 讀取.tfrecord的代碼是參考此文章：https://zhuanlan.zhihu.com/p/38575723

## 讀取tfrecord的範例

In [1]:
%matplotlib inline
import numpy as np
import tensorflow as tf

In [4]:
audio_record = 'C:/Users/admin/Downloads/features(1)/audioset_v1_embeddings/eval/_f.tfrecord'
vid_ids = [] #記錄此檔案中，包含的所有影片的ID
labels = [] #記錄此檔案中，每支影片的類別
start_time_seconds = [] # in secondes , 記錄此檔案中，每支影片的開始時間
end_time_seconds = [] # 記錄此檔案中，每支影片的結束時間
feat_audio = [] # 這些影片所提取出來的特徵值
count = 0
for example in tf.python_io.tf_record_iterator(audio_record):
    tf_example = tf.train.Example.FromString(example)
    #print(tf_example)
    vid_ids.append(tf_example.features.feature['video_id'].bytes_list.value[0].decode(encoding='UTF-8'))
    labels.append(tf_example.features.feature['labels'].int64_list.value)
    start_time_seconds.append(tf_example.features.feature['start_time_seconds'].float_list.value)
    end_time_seconds.append(tf_example.features.feature['end_time_seconds'].float_list.value)

    tf_seq_example = tf.train.SequenceExample.FromString(example)
    n_frames = len(tf_seq_example.feature_lists.feature_list['audio_embedding'].feature)

    sess = tf.InteractiveSession()
    rgb_frame = []
    audio_frame = []
    # iterate through frames
    for i in range(n_frames):
        audio_frame.append(tf.cast(tf.decode_raw(
                tf_seq_example.feature_lists.feature_list['audio_embedding'].feature[i].bytes_list.value[0],tf.uint8)
                       ,tf.float32).eval())

    sess.close()
    feat_audio.append([])

    feat_audio[count].append(audio_frame)
    count+=1


In [5]:
print(vid_ids)
print(labels)
print(start_time_seconds)
print(end_time_seconds)

['_FeZED3dwBs', '_F3mcLocpMw', '_FSBlSkqr1s', '_Fpn8NMnYNg', '_FoakdEOduA']
[[0, 453], [427, 431], [0, 59], [441], [366]]
[[30.0], [30.0], [130.0], [80.0], [30.0]]
[[40.0], [40.0], [140.0], [90.0], [40.0]]


In [9]:
print(feat_audio)

[[array([215., 255., 214.,  55.,  84., 158., 160., 155.,  15., 250., 119.,
       136.,  31., 166., 170., 173., 110., 156., 249., 202., 185.,  52.,
       127., 171., 219., 143.,   0., 150., 153., 144., 106., 174., 255.,
       253., 161.,  25., 192., 255., 104., 244.,  43., 112., 194., 255.,
       163., 149.,  32., 119., 216.,   0.,  84., 134.,  60.,  53., 255.,
       227.,  56.,  55., 169., 146.,  76.,  45.,  89.,  88., 188.,   0.,
        34., 122.,   0.,  89.,   0.,   0.,  98., 119., 138., 224., 105.,
       163., 235., 202., 107., 233., 172., 108., 119., 255.,   0.,  84.,
       255., 142., 119.,  61.,  90.,  37., 135., 186., 218., 167.,   0.,
       223.,  45.,  92.,   0., 173.,   0., 255., 105., 233., 247.,  12.,
       240., 168.,  54.,   0., 155., 219., 255.,   0., 255., 239., 237.,
         0.,  48., 191., 185.,   0.,  37., 172.], dtype=float32), array([212., 213., 205., 108., 134., 162., 147.,  28.,  46., 255., 124.,
       113.,  48., 255.,  55., 166., 150., 143., 255., 2